# MASTER - Data Merge

### Data Import 
<font color='red'>- set file paths below

-------------------------------------------------------------------------------------------------------------------------

In [5]:
import pandas as pd
import os

# VIIRS Satellite Fire .csv files path
F_path = r'C:\Datasets\data\fire_VIIRS_satellite_raw'

# Weather .csv files path
W_path = r'C:\Datasets\data\weather_raw_latlong'

import_path = r'C:\Datasets\data'
export_path = r'C:\Datasets\data\EXPORTS'

-------------------------------------------------------------------------------------------------------------------------

---
## Fire Satellite Data
---
<br/>Data has been acquired from year by year NASA VIIRS satellite data - https://firms2.modaps.eosdis.nasa.gov/country/
<br/>The key value in this data is frp, which is the fire radiative power detected in 375m gridded squares by the VIIRS satellite

<font color='red'>**NOTE!** 
<br/>- edit file paths (Cell 1 above) before running. 
<br/>- ensure all required data has been copied from MS Teams\Files\Dataset and stored locally
<br/>- WARNING, large data set

In [2]:
import glob

# F_path = r'C:\CP\UNI\S2\jupyter\SIT764_jupyter\data\VIIRS_375m'
all_files = glob.glob(F_path + "/*.csv")

# extracts data from each Satellite .csv file from path above
li = []
print(all_files)
# each .csv file is read and written to file called df, then each .csv is appended to list called li
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

# compiles .csv list data into dataframe
viirs = pd.concat(li, axis=0, ignore_index=True)

# more recent satellite .csv files do not have 'type' data - filling NaN values from these files with 0 (possibly inaccurate)
viirs['type'] = viirs['type'].fillna(0)

# reatining only 'confidence' values that don't equal 'l' (low). n (nominal) and h (high) values retained
viirs = viirs[(viirs[['confidence']] != 'l').all(axis=1)]

# retaining only measurement type 0 (presumed vegetation fire)
viirs = viirs[(viirs[['type']] == 0).all(axis=1)]

print('Measurement Confidence',viirs['confidence'].value_counts(),'\n',sep = '\n')
print('Measurement TYPE counts',viirs['type'].value_counts(),'\n',sep = '\n')
# setting date to datetime format
viirs['acq_date'] = pd.to_datetime(viirs.acq_date)
print(viirs.info())

# dropping unnecessary columns
viirs = viirs.drop(['scan','track','satellite','instrument','version','type'], axis=1)
# reorder columns
viirs = viirs[['acq_date','acq_time','latitude','longitude','frp','confidence','bright_ti4','bright_ti5','daynight']].reset_index(drop=True)

# trimming down the data set for reverse geocoding
viirs_filter = pd.DataFrame(viirs)
viirs_filter = viirs.filter(["latitude","longitude","acq_date","frp"])

# all data
viirs

['C:\\Datasets\\data\\fire_VIIRS_satellite_raw\\fire_nrt_J1V-C2_14894.csv', 'C:\\Datasets\\data\\fire_VIIRS_satellite_raw\\fire_nrt_J1V-C2_18000.csv', 'C:\\Datasets\\data\\fire_VIIRS_satellite_raw\\viirs-snpp_2014_Australia.csv', 'C:\\Datasets\\data\\fire_VIIRS_satellite_raw\\viirs-snpp_2015_Australia.csv', 'C:\\Datasets\\data\\fire_VIIRS_satellite_raw\\viirs-snpp_2016_Australia.csv', 'C:\\Datasets\\data\\fire_VIIRS_satellite_raw\\viirs-snpp_2017_Australia.csv', 'C:\\Datasets\\data\\fire_VIIRS_satellite_raw\\viirs-snpp_2018_Australia.csv', 'C:\\Datasets\\data\\fire_VIIRS_satellite_raw\\viirs-snpp_2019_Australia.csv', 'C:\\Datasets\\data\\fire_VIIRS_satellite_raw\\viirs-snpp_2020a_Australia.csv', 'C:\\Datasets\\data\\fire_VIIRS_satellite_raw\\viirs-snpp_2020b_Australia.csv']


C:\Users\vvanga\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


Measurement Confidence
n    6857680
h     764799
Name: confidence, dtype: int64


Measurement TYPE counts
0.0    7622479
Name: type, dtype: int64


<class 'pandas.core.frame.DataFrame'>
Int64Index: 7622479 entries, 0 to 8562084
Data columns (total 17 columns):
acq_date      datetime64[ns]
acq_time      int64
bright_t31    float64
bright_ti4    float64
bright_ti5    float64
brightness    float64
confidence    object
daynight      object
frp           float64
instrument    object
latitude      float64
longitude     float64
satellite     object
scan          float64
track         float64
type          float64
version       object
dtypes: datetime64[ns](1), float64(10), int64(1), object(5)
memory usage: 1.0+ GB
None


,acq_date,acq_time,latitude,longitude,frp,confidence,bright_ti4,bright_ti5,daynight
0,2020-12-01,242,-31.16067,152.85376,5.2,n,NaN,NaN,D
1,2020-12-01,242,-31.16096,152.84903,5.2,n,NaN,NaN,D
2,2020-12-01,242,-34.46362,150.88380,5.9,n,NaN,NaN,D
3,2020-12-01,242,-32.45356,152.42003,8.2,n,NaN,NaN,D
4,2020-12-01,242,-27.79847,148.68875,4.9,n,NaN,NaN,D
...,...,...,...,...,...,...,...,...,...
7622474,2020-12-09,1724,-33.21840,115.74820,1.0,n,304.4,291.8,N
7622475,2020-12-09,1724,-30.86475,121.49351,1.0,n,305.1,289.3,N
7622476,2020-12-09,1724,-30.86423,121.48860,1.0,n,301.8,288.9,N
7622477,2020-12-09,1724,-32.14737,115.79789,1.0,n,305.2,293.0,N


In [3]:
viirs['acq_date']

0         2020-12-01
1         2020-12-01
2         2020-12-01
3         2020-12-01
4         2020-12-01
             ...    
7622474   2020-12-09
7622475   2020-12-09
7622476   2020-12-09
7622477   2020-12-09
7622478   2020-12-09
Name: acq_date, Length: 7622479, dtype: datetime64[ns]

In [4]:
!pip install reverse_geocoder

  Created wheel for reverse-geocoder: filename=reverse_geocoder-1.5.1-cp37-none-any.whl size=2268089 sha256=45956f7a6c63c2c0a20cb0c68f0b808cc46bb67f2c00b00208dfc32b3d586b14
  Stored in directory: C:\Users\vvanga\AppData\Local\pip\Cache\wheels\47\05\50\b1350ff094ef91e082665b4a2f9ca551f8acea4aa55d796b26
Successfully built reverse-geocoder


### Creating reverse geocode suburb list from satellite data

In [4]:
# Adds locality name to lat long values 
import reverse_geocoder as rg
import pprint as pp

reverse_geo = viirs_filter[['latitude','longitude']].apply(tuple, axis=1)
reverse_geo = reverse_geo.tolist()
    
results = rg.search(reverse_geo)
rg_viirs = pd.DataFrame(results)

# tidying up the columns
rg_viirs.columns = ['lat_suburb','long_suburb','suburb','state','region','country']
rg_viirs = rg_viirs[['lat_suburb','long_suburb','suburb','region','state','country']]
rg_viirs

Loading formatted geocoded file...


,lat_suburb,long_suburb,suburb,region,state,country
0,-31.07898,152.83093,Kempsey,Kempsey,New South Wales,AU
1,-31.07898,152.83093,Kempsey,Kempsey,New South Wales,AU
2,-34.46667,150.88333,Cringila,Wollongong,New South Wales,AU
3,-32.40698,152.21185,Bulahdelah,Great Lakes,New South Wales,AU
4,-26.56741,148.78751,Roma,Maranoa,Queensland,AU
...,...,...,...,...,...,...
7622474,-33.2792,115.71504,Australind,Harvey,Western Australia,AU
7622475,-30.78204,121.4912,Boulder,Kalgoorlie/Boulder,Western Australia,AU
7622476,-30.78204,121.4912,Boulder,Kalgoorlie/Boulder,Western Australia,AU
7622477,-32.13339,115.80637,Beeliar,City of Cockburn,Western Australia,AU


### Merging reverse geocode suburb dataframe with satellite dataframe
Any measurements that aren't in Australia are dropped, fire data trimmed to match weather date range

In [5]:
fire = pd.concat([viirs,rg_viirs], axis=1, ignore_index=True)
fire.columns =['acq_date','acq_time','latitude','longitude','frp','confidence','bright_ti4','bright_ti5','daynight','lat_suburb','long_suburb','suburb','region','state','country']
fire.iloc[:,9:11] = fire.iloc[:,9:11].astype('float')
fire = fire[(fire[['country']] == 'AU').all(axis=1)].reset_index(drop=True)
fire = fire.drop(['country'], axis=1)

# filtering to match weather date range (2014-09-01 - 2020-11-30)
fire = fire.loc[fire["acq_date"] > "2014-08-31"]
fire = fire.loc[fire["acq_date"] < "2021-08-02"].reset_index(drop=True)
fire

,acq_date,acq_time,latitude,longitude,frp,confidence,bright_ti4,bright_ti5,daynight,lat_suburb,long_suburb,suburb,region,state
0,2020-12-01,242,-31.16067,152.85376,5.2,n,NaN,NaN,D,-31.07898,152.83093,Kempsey,Kempsey,New South Wales
1,2020-12-01,242,-31.16096,152.84903,5.2,n,NaN,NaN,D,-31.07898,152.83093,Kempsey,Kempsey,New South Wales
2,2020-12-01,242,-34.46362,150.88380,5.9,n,NaN,NaN,D,-34.46667,150.88333,Cringila,Wollongong,New South Wales
3,2020-12-01,242,-32.45356,152.42003,8.2,n,NaN,NaN,D,-32.40698,152.21185,Bulahdelah,Great Lakes,New South Wales
4,2020-12-01,242,-27.79847,148.68875,4.9,n,NaN,NaN,D,-26.56741,148.78751,Roma,Maranoa,Queensland
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7100957,2020-12-09,1724,-33.21840,115.74820,1.0,n,304.4,291.8,N,-33.27920,115.71504,Australind,Harvey,Western Australia
7100958,2020-12-09,1724,-30.86475,121.49351,1.0,n,305.1,289.3,N,-30.78204,121.49120,Boulder,Kalgoorlie/Boulder,Western Australia
7100959,2020-12-09,1724,-30.86423,121.48860,1.0,n,301.8,288.9,N,-30.78204,121.49120,Boulder,Kalgoorlie/Boulder,Western Australia
7100960,2020-12-09,1724,-32.14737,115.79789,1.0,n,305.2,293.0,N,-32.13339,115.80637,Beeliar,City of Cockburn,Western Australia


### Suburbs with missing region value

In [6]:
temp = fire[fire.region=='']  
#temp = fire[fire.region.isnull()]    # use this if fire data has been imported from a CSV rather than built from rg_viirs merge
temp = temp.groupby('suburb')
temp.count()

,acq_date,acq_time,latitude,longitude,frp,confidence,bright_ti4,bright_ti5,daynight,lat_suburb,long_suburb,region,state
suburb,,,,,,,,,,,,,
Acton,3,3,3,3,3,3,3,3,3,3,3,3,3
Alexandria,3,3,3,3,3,3,3,3,3,3,3,3,3
Alyangula,373375,373375,373375,373375,373375,373375,353955,353955,373375,373375,373375,373375,373375
Ballarat East,2,2,2,2,2,2,2,2,2,2,2,2,2
Belconnen,12,12,12,12,12,12,12,12,12,12,12,12,12
Black Hill,9,9,9,9,9,9,8,8,9,9,9,9,9
Booker Bay,2,2,2,2,2,2,2,2,2,2,2,2,2
Brown Hill,388,388,388,388,388,388,353,353,388,388,388,388,388
Canberra,1,1,1,1,1,1,1,1,1,1,1,1,1


#### Correcting records with NaN for region

In [7]:
import warnings
warnings.filterwarnings("ignore")

fire.region[fire.state == 'Australian Capital Territory'] = 'Australian Capital Territory'

fire.region[fire.suburb == 'Ballarat East'] = 'Ballarat'
fire.region[fire.suburb == 'Black Hill'] = 'Ballarat'
fire.region[fire.suburb == 'Brown Hill'] = 'Ballarat'
fire.region[fire.suburb == 'Heatherton'] = 'Kingston'
fire.region[fire.suburb == 'Mount Pleasant'] = 'Ballarat'
fire.region[fire.suburb == 'Plenty'] = 'Nillumbik'
fire.region[fire.suburb == 'Somers'] = 'Mornington Peninsula'
fire.region[fire.suburb == 'Alexandria'] = 'City of Sydney'
fire.region[fire.suburb == 'Booker Bay'] = 'Central Coast'
fire.region[fire.suburb == 'Chinderah'] = 'Tweed'
fire.region[fire.suburb == 'Alyangula'] = 'East Arnhem'
fire.region[fire.suburb == 'Port Denison'] = 'Irwin'

# assigning 'nil' to any records where there is no council defined by state government 

# fire['region'] = fire.region.fillna('nil')  - if data is imported from CSV - this method is required - blanks in dataframe have a value of NaN, not blank
fire['region'] = fire['region'].replace({'': 'nil'})

### Suburbs with missing region value - after fix 
These suburbs have no council region - they are in regional South Australia - council is undefined by the state government

In [8]:
temp1 = fire[fire.region == 'nil']
temp1 = temp1.groupby('suburb')
print('\nCount of records with value of nil for region = ',len(fire[fire.region == 'nil'].index),'\n')
temp1.first()


Count of records with value of nil for region =  5477 



,acq_date,acq_time,latitude,longitude,frp,confidence,bright_ti4,bright_ti5,daynight,lat_suburb,long_suburb,region,state
suburb,,,,,,,,,,,,,
Heidelberg West,2021-01-17,300,-37.749920,145.028780,3.20,n,NaN,NaN,D,-37.75000,145.03333,nil,Victoria
Innamincka,2020-12-03,1500,-24.704890,140.363070,2.90,n,354.79,311.94,N,-27.70728,140.73697,nil,South Australia
Koomooloo,2015-03-04,439,-33.883419,139.181046,3.47,n,346.75,318.21,D,-33.55470,139.46805,nil,South Australia
The Sill,2021-01-25,1512,-24.883160,137.818940,3.30,n,295.29,282.90,N,-28.16048,138.67540,nil,South Australia


### Create dataframe of all suburb geospatial info from fire dataset
(used in later code to add geospatial values to weather_fire)

In [9]:
fire_locats = fire.drop_duplicates(subset=['suburb', 'state'], keep='last').reset_index(drop=True)
fire_locats = fire_locats[['lat_suburb', 'long_suburb','suburb', 'region', 'state']]
fire_locats = fire_locats.sort_values(['suburb', 'state'], ascending=[True, True]).reset_index(drop=True)
fire_locats

,lat_suburb,long_suburb,suburb,region,state
0,-35.27767,149.11829,Acton,Australian Capital Territory,Australian Capital Territory
1,-34.91119,138.70735,Adelaide Hills,Adelaide Hills,South Australia
2,-35.00310,117.86595,Albany,Albany,Western Australia
3,-34.85925,138.52138,Alberton,Port Adelaide Enfield,South Australia
4,-35.01667,138.73333,Aldgate,Adelaide Hills,South Australia
...,...,...,...,...,...
1102,-23.12683,150.74406,Yeppoon,Rockhampton,Queensland
1103,-31.88809,116.76780,York,York,Western Australia
1104,-16.80278,145.72083,Yorkeys Knob,Cairns,Queensland
1105,-34.31350,148.30107,Young,Young,New South Wales


### Export FIRE_LOCATS

In [10]:
# EXPORT to CSV
file = 'fire_locats.csv'
vars()[file[0:-4]].to_csv('\\'.join([export_path,file[:]]),index = False)

### Export FIRE - large file - slow

In [11]:
# EXPORT to CSV
file = 'fire.csv'
vars()[file[0:-4]].to_csv('\\'.join([export_path,file[:]]),index = False)

---
---
---
## Weather Data
Merging individual weather files from    **<u><a href="https://www.worldweatheronline.com/developer/api/historical-weather-api.aspx" target="_blank">worldweatheronline.com**</a>.
<br/>All data files have already been acquired and have been uploaded to **MS Teams\Files\T3 2020\Dataset** or **bitbucket\datasets**.
<br/>See code file **T3_2020_Master_weather_download.ipynb** for code to acquire data, if required.

## !NOTE! - SUBURB.csv DATA NAMES ONLY
use the code below only if weather data CSVs are named by suburb name. If named by lat_long numbers - use the proceeding code

In [1]:
# import glob

# # W_path = r'C:\CP\UNI\S2\jupyter\SIT764_jupyter\data\weather'
# all_files = glob.glob(W_path + "/*.csv")

# # extracts data from each .csv file from path above
# li = []

# for filename in all_files:
#     df = pd.read_csv(filename, skiprows=[0], index_col=None, header=None)
#     li.append(df)

# # compiles .csv list data into dataframe
# weather = pd.concat(li, axis=0, ignore_index=True)
# weather.columns =['date','maxC','minC','Snow_cm','sunHour','uv','moon_illum','moonrise','moonset','sunrise','sunset','DewPointC','FeelsLikeC','HeatIndexC','WindChillC','WindGustKmph','cloudcover','humidity','precipMM','pressure','tempC','visibility','windAZI','windKmph','location']   
# weather = weather.drop(['moon_illum','moonrise','moonset','sunrise','sunset'], axis=1)

# # removing %20 and state,Australia from weather files with spaces 
# #      e.g Adelaide%20Hills,South%20Australia,Australia to Adelaide Hills)
# weather['location']= weather['location'].str.replace("%20", " ", case = False) 
# weather['location'] = weather['location'].str.split(',').str[0]

# # setting the date field as a datetime field
# weather['date'] = pd.to_datetime(weather.date)

# weather

## !NOTE! - LAT,LONG.csv FILE NAMES ONLY
use the code below only if weather data CSVs are named by lat,long (e.g. -35.05, 143.55.csv). If named by suburb name - use the prior code

In [6]:
import glob

# W_path = r'C:\CP\UNI\jupyter\Jpt_2021_S1\SIT782\data\weather\2021_T1\745_1094'
all_files = glob.glob(W_path + "/*.csv")

# extracts data from each .csv file from path above
li = []

for filename in all_files:
    df = pd.read_csv(filename, skiprows=[0], index_col=None, header=None)
    li.append(df)

# compiles .csv list data into dataframe
weather = pd.concat(li, axis=0, ignore_index=True)
weather.columns =['date','maxC','minC','Snow_cm','sunHour','uv','moon_illum','moonrise','moonset','sunrise',
                  'sunset','DewPointC','FeelsLikeC','HeatIndexC','WindChillC','WindGustKmph','cloudcover',
                  'humidity','precipMM','pressure','tempC','visibility','windAZI','windKmph','location']   


'''# SUBURB - this code is required if the data files are named by SUBURB - 
cleans out %20 characters from file nameremoving %20 and state,Australia from weather files with spaces 
e.g Adelaide%20Hills,South%20Australia,Australia to Adelaide Hills)'''
# weather['location']= weather['location'].str.replace("%20", " ", case = False)

# LAT_LONG data
# run this code if the weather data is collected by LAT_LONG location method - code splits the location string into lat and long columns and converts to float
weather['lat_weather'] = weather['location'].str.split(',').str[0].astype('float')
weather['long_weather'] = weather['location'].str.split(',').str[1].astype('float')

weather = weather.drop(['moon_illum','moonrise','moonset','sunrise','sunset','location'], axis=1)

# setting the date field as a datetime field
weather['date'] = pd.to_datetime(weather.date)

weather

,date,maxC,minC,Snow_cm,sunHour,uv,DewPointC,FeelsLikeC,HeatIndexC,WindChillC,...,cloudcover,humidity,precipMM,pressure,tempC,visibility,windAZI,windKmph,lat_weather,long_weather
0,2021-05-01,27,23,0.0,5.9,6,23,29,29,26,...,69,83,5.3,1012,27,10,120,18,-10.57937,142.21686
1,2021-05-02,27,24,0.0,8.8,6,23,29,29,26,...,61,84,8.0,1011,27,10,123,19,-10.57937,142.21686
2,2021-05-03,28,26,0.0,5.9,6,24,30,30,27,...,66,84,10.4,1009,28,10,124,18,-10.57937,142.21686
3,2021-05-04,28,24,0.0,7.3,6,24,30,30,26,...,70,84,12.6,1009,28,9,125,21,-10.57937,142.21686
4,2021-05-05,27,26,0.0,7.3,6,24,30,30,27,...,65,86,12.5,1009,27,9,130,22,-10.57937,142.21686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545014,2021-07-29,9,4,0.0,3.7,2,3,4,7,4,...,65,78,3.2,999,9,9,293,14,-43.16337,146.92549
545015,2021-07-30,13,4,0.0,7.0,2,3,6,8,6,...,54,71,0.0,1007,13,10,270,14,-43.16337,146.92549
545016,2021-07-31,13,8,0.0,3.7,3,6,9,11,9,...,74,73,2.1,999,13,8,295,15,-43.16337,146.92549
545017,2021-08-01,10,5,0.0,5.4,2,3,6,8,6,...,36,71,0.3,1004,10,10,287,13,-43.16337,146.92549


In [7]:
# weather['location1'] = weather['lat_long'].str.split(',').str[1]
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545019 entries, 0 to 545018
Data columns (total 21 columns):
date            545019 non-null datetime64[ns]
maxC            545019 non-null int64
minC            545019 non-null int64
Snow_cm         545019 non-null float64
sunHour         545019 non-null float64
uv              545019 non-null int64
DewPointC       545019 non-null int64
FeelsLikeC      545019 non-null int64
HeatIndexC      545019 non-null int64
WindChillC      545019 non-null int64
WindGustKmph    545019 non-null int64
cloudcover      545019 non-null int64
humidity        545019 non-null int64
precipMM        545019 non-null float64
pressure        545019 non-null int64
tempC           545019 non-null int64
visibility      545019 non-null int64
windAZI         545019 non-null int64
windKmph        545019 non-null int64
lat_weather     545019 non-null float64
long_weather    545019 non-null float64
dtypes: datetime64[ns](1), float64(5), int64(15)
memory usage: 87.3 MB


In [8]:
weather_locats = weather.drop_duplicates(subset=['lat_weather', 'long_weather'], keep='last').reset_index(drop=True)
weather_locats = weather_locats[['lat_weather', 'long_weather']]
weather_locats = weather_locats.sort_values(['lat_weather', 'long_weather'], ascending=[True, True]).reset_index(drop=True)
weather_locats

,lat_weather,long_weather
0,-43.16337,146.92549
1,-43.15333,147.07253
2,-43.12623,147.24641
3,-43.08884,147.00906
4,-43.03124,147.04813
...,...,...
1089,-12.42032,130.85506
1090,-12.38299,130.85170
1091,-12.36667,130.90000
1092,-12.18652,136.78201


In [9]:
# Adds locality name to lat long values 
import reverse_geocoder as rg
import pprint as pp

weather_revgeo = weather[['lat_weather','long_weather']].apply(tuple, axis=1)
weather_revgeo = weather_revgeo.tolist()
    
results1 = rg.search(weather_revgeo)
rg_weather = pd.DataFrame(results1)

# tidying up the columns
rg_weather.columns = ['lat_w','long_w','suburb','state','region','country']
rg_weather = rg_weather[['lat_w','long_w','suburb','region','state','country']]
rg_weather

Loading formatted geocoded file...


,lat_w,long_w,suburb,region,state,country
0,-10.57937,142.21686,Thursday Island,Torres,Queensland,AU
1,-10.57937,142.21686,Thursday Island,Torres,Queensland,AU
2,-10.57937,142.21686,Thursday Island,Torres,Queensland,AU
3,-10.57937,142.21686,Thursday Island,Torres,Queensland,AU
4,-10.57937,142.21686,Thursday Island,Torres,Queensland,AU
...,...,...,...,...,...,...
545014,-43.16337,146.92549,Geeveston,Huon Valley,Tasmania,AU
545015,-43.16337,146.92549,Geeveston,Huon Valley,Tasmania,AU
545016,-43.16337,146.92549,Geeveston,Huon Valley,Tasmania,AU
545017,-43.16337,146.92549,Geeveston,Huon Valley,Tasmania,AU


In [10]:
weather = pd.concat([weather,rg_weather], axis=1, ignore_index=True)
weather.columns =['date', 'maxC', 'minC','Snow_cm', 'sunHour', 'uv', 'DewPointC', 'FeelsLikeC',
                  'HeatIndexC', 'WindChillC', 'WindGustKmph', 'cloudcover', 'humidity',
                  'precipMM', 'pressure','tempC', 'visibility', 'windAZI', 'windKmph',
                  'lat_weather', 'long_weather','lat_w', 'long_w','suburb','region','state','country']

weather = weather[(weather[['country']] == 'AU').all(axis=1)].reset_index(drop=True)
weather = weather.drop(['country'], axis=1)
weather = weather.sort_values(['suburb', 'date'], ascending=[True, True]).reset_index(drop=True)

weather

,date,maxC,minC,Snow_cm,sunHour,uv,DewPointC,FeelsLikeC,HeatIndexC,WindChillC,...,visibility,windAZI,windKmph,lat_weather,long_weather,lat_w,long_w,suburb,region,state
0,2014-09-01,16,2,0.0,8.7,3,5,7,9,7,...,6,306,12,-35.27767,149.11829,-35.27767,149.11829,Acton,,Australian Capital Territory
1,2014-09-02,11,3,0.0,7.2,2,4,4,6,4,...,8,248,11,-35.27767,149.11829,-35.27767,149.11829,Acton,,Australian Capital Territory
2,2014-09-03,11,0,0.0,8.7,3,-1,1,4,1,...,10,202,13,-35.27767,149.11829,-35.27767,149.11829,Acton,,Australian Capital Territory
3,2014-09-04,13,-2,0.0,8.7,2,0,3,5,3,...,10,182,9,-35.27767,149.11829,-35.27767,149.11829,Acton,,Australian Capital Territory
4,2014-09-05,13,0,0.0,8.7,2,2,4,6,4,...,7,162,9,-35.27767,149.11829,-35.27767,149.11829,Acton,,Australian Capital Territory
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545014,2021-07-29,22,14,0.0,7.2,4,8,17,18,17,...,10,269,17,-27.35591,153.04453,-27.35591,153.04453,Zillmere,Brisbane,Queensland
545015,2021-07-30,19,12,0.0,8.7,5,6,15,16,15,...,10,131,11,-27.35591,153.04453,-27.35591,153.04453,Zillmere,Brisbane,Queensland
545016,2021-07-31,20,12,0.0,8.7,5,10,16,16,16,...,10,183,10,-27.35591,153.04453,-27.35591,153.04453,Zillmere,Brisbane,Queensland
545017,2021-08-01,23,14,0.0,8.7,5,13,18,19,18,...,10,201,10,-27.35591,153.04453,-27.35591,153.04453,Zillmere,Brisbane,Queensland


In [17]:
name = 'weather_locats.csv'
rg_weather.to_csv(os.path.join(export_path, name),index=False)

### Export WEATHER

In [18]:
file = 'weather.csv'
vars()[file[0:-4]].to_csv('\\'.join([export_path,file[:]]),index = False)

---
---
---
# Data Merge
---
<br/>Merging combined weather data with combined fire data
<br/>Combined files are available as .csv at **Teams\Files\T3 2020\Dataset** and **bitbucket\datasets**.

---
## Merge Method 1 - **FIRE_WEATHER**
Creates a linked dataset containing only records with fire (frp) values.
<br/> - for each suburb, any weather data days without fire data are not merged 
<br/> - some suburbs will show multiple frp values for one day. frp is measured in 375m grid squares across the suburb, any grid with fire detected is a row value

In [19]:
# Merging two datasets on 'date/acq_date' and 'location/suburb'
fire_weather = pd.merge(weather,fire, how='right',left_on=['date','suburb'],right_on=['acq_date','suburb'])

temp = pd.DataFrame(fire_weather.isnull().sum()).T
pd.set_option('display.max_columns', None)
print('Missing value Count - fire_weather. Total records = ',len(fire_weather))
temp

Missing value Count - fire_weather. Total records =  7110839


,date,maxC,minC,Snow_cm,sunHour,uv,DewPointC,FeelsLikeC,HeatIndexC,WindChillC,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,windAZI,windKmph,lat_weather,long_weather,lat_w,long_w,suburb,region_x,state_x,acq_date,acq_time,latitude,longitude,frp,confidence,bright_ti4,bright_ti5,daynight,lat_suburb,long_suburb,region_y,state_y
0,6767927,6767927,6767927,6767927,6767927,6767927,6767927,6767927,6767927,6767927,6767927,6767927,6767927,6767927,6767927,6767927,6767927,6767927,6767927,6767927,6767927,6767927,6767927,0,6767927,6767927,0,0,0,0,0,0,296273,296273,0,0,0,0,0


In [23]:
# drop missing data - (missing due there being no matching weather data suburb) 
fire_weather = fire_weather.dropna().reset_index(drop=True)
temp = pd.DataFrame(fire_weather.isnull().sum()).T
pd.set_option('display.max_columns', None)
print('Missing value Count after clean - fire_weather. Total records = ',len(fire_weather))
temp

Missing value Count after clean - fire_weather. Total records =  143389


,date,maxC,minC,Snow_cm,sunHour,uv,DewPointC,FeelsLikeC,HeatIndexC,WindChillC,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,windAZI,windKmph,lat_weather,long_weather,lat_w,long_w,suburb,region_x,state_x,acq_date,acq_time,latitude,longitude,frp,confidence,bright_ti4,bright_ti5,daynight,lat_suburb,long_suburb,region_y,state_y
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
pd.set_option('display.max_columns', 25)
fire_weather

,date,maxC,minC,Snow_cm,sunHour,uv,DewPointC,FeelsLikeC,HeatIndexC,WindChillC,WindGustKmph,cloudcover,...,acq_time,latitude,longitude,frp,confidence,bright_ti4,bright_ti5,daynight,lat_suburb,long_suburb,region_y,state_y
0,2017-04-07,22.0,5.0,0.0,8.7,4.0,4.0,12.0,13.0,12.0,6.0,13.0,...,348,-35.277077,149.105057,1.37,n,328.64,295.80,D,-35.27767,149.11829,Australian Capital Territory,Australian Capital Territory
1,2018-04-26,22.0,10.0,0.0,8.7,5.0,6.0,15.0,15.0,15.0,10.0,16.0,...,348,-35.260887,149.106171,1.47,n,326.79,301.04,D,-35.27767,149.11829,Australian Capital Territory,Australian Capital Territory
2,2018-05-02,20.0,10.0,0.0,8.7,5.0,4.0,14.0,14.0,14.0,8.0,33.0,...,1455,-35.263260,149.102097,0.82,n,302.81,281.51,N,-35.27767,149.11829,Australian Capital Territory,Australian Capital Territory
3,2016-05-13,21.0,12.0,0.0,8.7,4.0,10.0,15.0,16.0,15.0,16.0,12.0,...,459,-34.942699,138.738663,3.62,n,330.36,288.63,D,-34.91119,138.70735,Adelaide Hills,South Australia
4,2016-11-22,19.0,13.0,0.0,12.3,4.0,11.0,15.0,15.0,15.0,14.0,58.0,...,1559,-34.913586,138.740677,0.51,n,296.30,279.95,N,-34.91119,138.70735,Adelaide Hills,South Australia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143384,2015-03-05,30.0,21.0,0.0,11.6,7.0,12.0,26.0,26.0,25.0,29.0,2.0,...,1720,-32.132889,115.829643,0.61,n,306.36,292.54,N,-32.12065,115.81623,City of Cockburn,Western Australia
143385,2017-05-20,23.0,18.0,0.0,7.2,5.0,17.0,20.0,20.0,20.0,13.0,44.0,...,1459,-27.369436,153.068573,0.39,n,297.20,286.94,N,-27.35591,153.04453,Brisbane,Queensland
143386,2017-08-17,30.0,20.0,0.0,8.7,7.0,11.0,25.0,25.0,24.0,19.0,1.0,...,1531,-27.363457,153.069351,0.32,n,296.83,285.29,N,-27.35591,153.04453,Brisbane,Queensland
143387,2017-08-18,26.0,17.0,0.0,8.7,6.0,2.0,20.0,20.0,21.0,31.0,0.0,...,357,-27.362944,153.068832,9.41,n,353.08,303.08,D,-27.35591,153.04453,Brisbane,Queensland


### Export FIRE_WEATHER 
#### (slow process - 1.5GB output file)

In [25]:
# EXPORT to CSV
file = 'fire_weather.csv'
vars()[file[0:-4]].to_csv('\\'.join([export_path,file[:]]),index = False)

---
---
---
## Merge Method 2 - **WEATHER_FIRE**
Creates a linked dataset containing all days for every suburb in the date range
<br/> Days per suburb without frp are assigned a value of zero for all frp fields.
<br/> Several frp values are incorporated, for trialling in data analysis/prediction (frp_mean, frp_median, frp_min, frp_max, frp_sum)

In [26]:
# grouping data by date and suburb (retains all geospatial information)

fire_grouped = fire.groupby(['acq_date', 'lat_suburb', 'long_suburb','suburb','state','region']).agg({'frp': ['min', 'max', 'sum', 'median', 'mean']})
fire_grouped.columns = ['frp_min', 'frp_max', 'frp_sum', 'frp_median', 'frp_mean']

fire_grouped = fire_grouped.reset_index()
fire_grouped

,acq_date,lat_suburb,long_suburb,suburb,state,region,frp_min,frp_max,frp_sum,frp_median,frp_mean
0,2014-09-01,-41.15780,147.51727,Scottsdale,Tasmania,Dorset,2.81,2.81,2.81,2.810,2.810000
1,2014-09-01,-36.30768,140.77167,Bordertown,South Australia,Tatiara,2.57,2.68,5.25,2.625,2.625000
2,2014-09-01,-34.95778,117.93833,Lower King,Western Australia,Albany,6.33,10.23,16.56,8.280,8.280000
3,2014-09-01,-34.81826,138.96478,Birdwood,South Australia,Adelaide Hills,13.14,22.12,48.40,13.140,16.133333
4,2014-09-01,-34.18551,142.16251,Mildura,Victoria,Mildura Shire,0.91,1.13,2.04,1.020,1.020000
...,...,...,...,...,...,...,...,...,...,...,...
211254,2021-08-01,-12.46113,130.84185,Darwin,Northern Territory,Darwin,3.04,24.34,136.28,8.810,12.389091
211255,2021-08-01,-12.45527,130.83172,Larrakeyah,Northern Territory,Darwin,0.47,47.91,460.48,5.620,8.855385
211256,2021-08-01,-12.38299,130.85170,Nightcliff,Northern Territory,Darwin,0.49,15.65,78.20,5.780,5.585714
211257,2021-08-01,-12.36667,130.90000,Leanyer,Northern Territory,Darwin,0.55,19.86,277.94,5.020,5.053455


#### Merging weather and fire data (based on weather observations)

In [27]:
weather_fire = pd.merge(weather,fire_grouped, how='left',left_on=['date','suburb'],right_on=['acq_date','suburb'])
weather_fire

,date,maxC,minC,Snow_cm,sunHour,uv,DewPointC,FeelsLikeC,HeatIndexC,WindChillC,WindGustKmph,cloudcover,...,region_x,state_x,acq_date,lat_suburb,long_suburb,state_y,region_y,frp_min,frp_max,frp_sum,frp_median,frp_mean
0,2014-09-01,16,2,0.0,8.7,3,5,7,9,7,20,39,...,,Australian Capital Territory,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-09-02,11,3,0.0,7.2,2,4,4,6,4,17,65,...,,Australian Capital Territory,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-09-03,11,0,0.0,8.7,3,-1,1,4,1,20,17,...,,Australian Capital Territory,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014-09-04,13,-2,0.0,8.7,2,0,3,5,3,15,12,...,,Australian Capital Territory,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014-09-05,13,0,0.0,8.7,2,2,4,6,4,16,41,...,,Australian Capital Territory,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545270,2021-07-29,22,14,0.0,7.2,4,8,17,18,17,26,32,...,Brisbane,Queensland,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
545271,2021-07-30,19,12,0.0,8.7,5,6,15,16,15,16,4,...,Brisbane,Queensland,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
545272,2021-07-31,20,12,0.0,8.7,5,10,16,16,16,14,2,...,Brisbane,Queensland,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
545273,2021-08-01,23,14,0.0,8.7,5,13,18,19,18,17,3,...,Brisbane,Queensland,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Count of records with no data.
fire data null values are expected, fires do not occur everyday in every suburb

In [28]:
weather_fire.isnull().sum()

date                 0
maxC                 0
minC                 0
Snow_cm              0
sunHour              0
uv                   0
DewPointC            0
FeelsLikeC           0
HeatIndexC           0
WindChillC           0
WindGustKmph         0
cloudcover           0
humidity             0
precipMM             0
pressure             0
tempC                0
visibility           0
windAZI              0
windKmph             0
lat_weather          0
long_weather         0
lat_w                0
long_w               0
suburb               0
region_x             0
state_x              0
acq_date        524548
lat_suburb      524548
long_suburb     524548
state_y         524548
region_y        524548
frp_min         524548
frp_max         524548
frp_sum         524548
frp_median      524548
frp_mean        524548
dtype: int64

#### Replacing missing values

In [29]:
# replacing 'acq_date' and 'suburb' NaN values with corresponding values from 'location' and 'date'
weather_fire.acq_date.fillna(weather_fire.date, inplace=True) 
weather_fire.suburb.fillna(weather_fire.suburb, inplace=True)

# replacing frp NaN values with 0
weather_fire.iloc[:,21:31] = weather_fire.iloc[:,21:31].fillna(0)

weather_fire

,date,maxC,minC,Snow_cm,sunHour,uv,DewPointC,FeelsLikeC,HeatIndexC,WindChillC,WindGustKmph,cloudcover,...,region_x,state_x,acq_date,lat_suburb,long_suburb,state_y,region_y,frp_min,frp_max,frp_sum,frp_median,frp_mean
0,2014-09-01,16,2,0.0,8.7,3,5,7,9,7,20,39,...,,Australian Capital Territory,2014-09-01,0.0,0.0,0,0,NaN,NaN,NaN,NaN,NaN
1,2014-09-02,11,3,0.0,7.2,2,4,4,6,4,17,65,...,,Australian Capital Territory,2014-09-02,0.0,0.0,0,0,NaN,NaN,NaN,NaN,NaN
2,2014-09-03,11,0,0.0,8.7,3,-1,1,4,1,20,17,...,,Australian Capital Territory,2014-09-03,0.0,0.0,0,0,NaN,NaN,NaN,NaN,NaN
3,2014-09-04,13,-2,0.0,8.7,2,0,3,5,3,15,12,...,,Australian Capital Territory,2014-09-04,0.0,0.0,0,0,NaN,NaN,NaN,NaN,NaN
4,2014-09-05,13,0,0.0,8.7,2,2,4,6,4,16,41,...,,Australian Capital Territory,2014-09-05,0.0,0.0,0,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545270,2021-07-29,22,14,0.0,7.2,4,8,17,18,17,26,32,...,Brisbane,Queensland,2021-07-29,0.0,0.0,0,0,NaN,NaN,NaN,NaN,NaN
545271,2021-07-30,19,12,0.0,8.7,5,6,15,16,15,16,4,...,Brisbane,Queensland,2021-07-30,0.0,0.0,0,0,NaN,NaN,NaN,NaN,NaN
545272,2021-07-31,20,12,0.0,8.7,5,10,16,16,16,14,2,...,Brisbane,Queensland,2021-07-31,0.0,0.0,0,0,NaN,NaN,NaN,NaN,NaN
545273,2021-08-01,23,14,0.0,8.7,5,13,18,19,18,17,3,...,Brisbane,Queensland,2021-08-01,0.0,0.0,0,0,NaN,NaN,NaN,NaN,NaN


In [30]:
# adding lost geospatial data back into the merged table (lat_suburb, long_suburb, state, region)

weather_fire['lat_suburb'] = weather_fire['suburb']
weather_fire['long_suburb'] = weather_fire['suburb']
weather_fire['state'] = weather_fire['suburb']
weather_fire['region'] = weather_fire['suburb']

rename_dict1 = fire_locats.set_index('suburb').to_dict()['lat_suburb']
rename_dict2 = fire_locats.set_index('suburb').to_dict()['long_suburb']
rename_dict3 = fire_locats.set_index('suburb').to_dict()['state']
rename_dict4 = fire_locats.set_index('suburb').to_dict()['region']

weather_fire['lat_suburb'] = weather_fire['lat_suburb'].map(rename_dict1)
weather_fire['long_suburb'] = weather_fire['long_suburb'].map(rename_dict2)
weather_fire['state'] = weather_fire['state'].map(rename_dict3)
weather_fire['region'] = weather_fire['region'].map(rename_dict4)
weather_fire

,date,maxC,minC,Snow_cm,sunHour,uv,DewPointC,FeelsLikeC,HeatIndexC,WindChillC,WindGustKmph,cloudcover,...,acq_date,lat_suburb,long_suburb,state_y,region_y,frp_min,frp_max,frp_sum,frp_median,frp_mean,state,region
0,2014-09-01,16,2,0.0,8.7,3,5,7,9,7,20,39,...,2014-09-01,-35.27767,149.11829,0,0,NaN,NaN,NaN,NaN,NaN,Australian Capital Territory,Australian Capital Territory
1,2014-09-02,11,3,0.0,7.2,2,4,4,6,4,17,65,...,2014-09-02,-35.27767,149.11829,0,0,NaN,NaN,NaN,NaN,NaN,Australian Capital Territory,Australian Capital Territory
2,2014-09-03,11,0,0.0,8.7,3,-1,1,4,1,20,17,...,2014-09-03,-35.27767,149.11829,0,0,NaN,NaN,NaN,NaN,NaN,Australian Capital Territory,Australian Capital Territory
3,2014-09-04,13,-2,0.0,8.7,2,0,3,5,3,15,12,...,2014-09-04,-35.27767,149.11829,0,0,NaN,NaN,NaN,NaN,NaN,Australian Capital Territory,Australian Capital Territory
4,2014-09-05,13,0,0.0,8.7,2,2,4,6,4,16,41,...,2014-09-05,-35.27767,149.11829,0,0,NaN,NaN,NaN,NaN,NaN,Australian Capital Territory,Australian Capital Territory
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545270,2021-07-29,22,14,0.0,7.2,4,8,17,18,17,26,32,...,2021-07-29,-27.35591,153.04453,0,0,NaN,NaN,NaN,NaN,NaN,Queensland,Brisbane
545271,2021-07-30,19,12,0.0,8.7,5,6,15,16,15,16,4,...,2021-07-30,-27.35591,153.04453,0,0,NaN,NaN,NaN,NaN,NaN,Queensland,Brisbane
545272,2021-07-31,20,12,0.0,8.7,5,10,16,16,16,14,2,...,2021-07-31,-27.35591,153.04453,0,0,NaN,NaN,NaN,NaN,NaN,Queensland,Brisbane
545273,2021-08-01,23,14,0.0,8.7,5,13,18,19,18,17,3,...,2021-08-01,-27.35591,153.04453,0,0,NaN,NaN,NaN,NaN,NaN,Queensland,Brisbane


In [31]:
print('','Records per State',weather_fire['state'].value_counts(),sep='\n')
print('','Counts of Missing values',weather_fire.isnull().sum(),sep='\n')


Records per State
Queensland                      127607
New South Wales                 125922
Western Australia               117270
Victoria                        114145
South Australia                  30730
Tasmania                         15412
Australian Capital Territory     12873
Northern Territory                1316
Name: state, dtype: int64

Counts of Missing values
date                 0
maxC                 0
minC                 0
Snow_cm              0
sunHour              0
uv                   0
DewPointC            0
FeelsLikeC           0
HeatIndexC           0
WindChillC           0
WindGustKmph         0
cloudcover           0
humidity             0
precipMM             0
pressure             0
tempC                0
visibility           0
windAZI              0
windKmph             0
lat_weather          0
long_weather         0
lat_w                0
long_w               0
suburb               0
region_x             0
state_x              0
acq_date           

In [32]:
# dropping missing Na values
print("Shape before Na's dropped\n",weather_fire.shape)
weather_fire = weather_fire.dropna()
print("Shape after Na's dropped\n",weather_fire.shape)

Shape before Na's dropped
 (545275, 38)
Shape after Na's dropped
 (20727, 38)


### <u>Export WEATHER_FIRE</u>

In [37]:
# EXPORT to CSV
file = 'weather_fire.csv'
vars()[file[0:-4]].to_csv('\\'.join([export_path,file[:]]),index = False)

---
---
---

## Merge Method 3 - **WEATHER_FIRE_REGION**
Creates a linked dataset containing all days for every region in the date range
<br/> Days per region without frp are assigned a value of zero for all frp fields.
<br/> Several frp values are incorporated, for trialling in data analysis/prediction (frp_mean, frp_median, frp_min, frp_max, frp_sum)

**Files required**
- fire
- weather
- fire_locats

### Grouping fire data by date and region 
this causes loss of suburb name and suburb lat & long

In [38]:
df = fire.copy()
fire.iloc[:,9:11] = fire.iloc[:,9:11].astype('float')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7100962 entries, 0 to 7100961
Data columns (total 14 columns):
acq_date       datetime64[ns]
acq_time       int64
latitude       float64
longitude      float64
frp            float64
confidence     object
bright_ti4     float64
bright_ti5     float64
daynight       object
lat_suburb     float64
long_suburb    float64
suburb         object
region         object
state          object
dtypes: datetime64[ns](1), float64(7), int64(1), object(5)
memory usage: 758.5+ MB


In [39]:
fire_reg_grouped = fire.groupby(['region','state','acq_date']).agg({'frp': ['min', 'max', 'sum', 'median', 'mean'],'lat_suburb':'mean','long_suburb':'mean'})
fire_reg_grouped.columns = ['frp_min', 'frp_max', 'frp_sum', 'frp_median', 'frp_mean','lat_ave','long_ave']
fire_reg_grouped = fire_reg_grouped.reset_index()
fire_reg_grouped

,region,state,acq_date,frp_min,frp_max,frp_sum,frp_median,frp_mean,lat_ave,long_ave
0,Adelaide Hills,South Australia,2014-09-01,13.14,22.12,48.40,13.140,16.133333,-34.81826,138.96478
1,Adelaide Hills,South Australia,2014-09-08,6.10,6.10,6.10,6.100,6.100000,-34.81826,138.96478
2,Adelaide Hills,South Australia,2014-09-20,0.20,0.20,0.20,0.200,0.200000,-35.00000,138.75000
3,Adelaide Hills,South Australia,2014-10-15,49.95,82.98,132.93,66.465,66.465000,-34.81826,138.96478
4,Adelaide Hills,South Australia,2014-10-30,4.31,4.31,4.31,4.310,4.310000,-35.01667,138.68333
...,...,...,...,...,...,...,...,...,...,...
174990,nil,South Australia,2021-07-03,4.58,4.58,4.58,4.580,4.580000,-27.70728,140.73697
174991,nil,South Australia,2021-07-24,8.05,8.05,8.05,8.050,8.050000,-27.70728,140.73697
174992,nil,South Australia,2021-07-26,0.43,1.66,6.75,0.830,0.964286,-27.70728,140.73697
174993,nil,South Australia,2021-07-29,0.59,0.59,0.59,0.590,0.590000,-27.70728,140.73697


### Creating lat and long dictionary for regions
suburb lat and longs within a region are averaged to a single lat and long value for the region

In [40]:
fire_reg_dict = fire_reg_grouped.groupby(['region','state']).agg({'lat_ave':'mean','long_ave':'mean'}).reset_index()
print(fire_reg_dict.shape)
fire_reg_dict.iloc[:4,]

(375, 4)


,region,state,lat_ave,long_ave
0,Adelaide Hills,South Australia,-34.876731,138.865255
1,Albany,Western Australia,-34.967134,117.913350
2,Albury Municipality,New South Wales,-36.068615,146.926806
3,Alexandrina,South Australia,-35.365570,138.796727


#### merge check - compare with values in table above

In [41]:
import warnings
warnings.filterwarnings("ignore")

print('Alexandrina =\n',fire_reg_grouped.lat_ave[fire_reg_grouped.region == 'Alexandrina'].mean())
print(fire_reg_grouped.long_ave[fire_reg_grouped.region == 'Alexandrina'].mean())

Alexandrina =
 -35.36556975208636
138.79672749386359


#### creating dictionary for region lat and region long

In [42]:
fire_reg_dict_lat = dict(zip(fire_reg_dict.region, fire_reg_dict.lat_ave))
fire_reg_dict_long = dict(zip(fire_reg_dict.region, fire_reg_dict.long_ave))

### Add Region & State to Weather data
- Create dictionaries of suburb:region and suburb:state from fire data

In [44]:
file = 'fire_locats.csv'
vars()[file[0:-4]] = pd.read_csv('\\'.join([import_path,file[:]]))

fire_locats_dict_reg = dict(zip(fire_locats.suburb, fire_locats.region))
fire_locats_dict_state = dict(zip(fire_locats.suburb, fire_locats.state))
print(file,' = ',fire_locats.shape)

fire_locats.csv  =  (1107, 5)


- Add region, state, lat_ave & long_ave to weather data

In [46]:
weather['region'] = weather['suburb'].map(fire_locats_dict_reg)
weather['state'] = weather['suburb'].map(fire_locats_dict_state)
weather['lat_ave'] = weather['region'].map(fire_reg_dict_lat)
weather['long_ave'] = weather['region'].map(fire_reg_dict_long)
weather[weather.suburb == 'Torquay']

,date,maxC,minC,Snow_cm,sunHour,uv,DewPointC,FeelsLikeC,HeatIndexC,WindChillC,WindGustKmph,cloudcover,...,visibility,windAZI,windKmph,lat_weather,long_weather,lat_w,long_w,suburb,region,state,lat_ave,long_ave
982,2017-03-14,30,26,0.0,11.6,6,23,32,32,28,7,28,...,9,213,4,-10.57937,142.21686,-25.28496,152.87886,Torquay,Fraser Coast,Queensland,-25.474071,152.746728


### Grouping weather data by region

In [47]:
weather_grouped = weather.groupby(['region','state','date','lat_ave','long_ave']).agg({'maxC': 'mean','minC': 'mean','uv': 'mean','humidity': 'mean','cloudcover': 'mean','precipMM': 'mean','pressure': 'mean','windAZI': 'mean','windKmph': 'mean','WindGustKmph': 'mean'})
weather_grouped.columns = ['maxC','minC','uv','humidity','cloudcover','precipMM','pressure','windAZI','windKmph','WindGustKmph']

weather_grouped = weather_grouped.reset_index()
weather_grouped

,region,state,date,lat_ave,long_ave,maxC,minC,uv,humidity,cloudcover,precipMM,pressure,windAZI,windKmph,WindGustKmph
0,Adelaide Hills,South Australia,2015-05-01,-34.876731,138.865255,28,26,6,85,42,52.1,1009,144,18,30
1,Adelaide Hills,South Australia,2015-05-02,-34.876731,138.865255,29,26,6,81,33,16.5,1009,129,15,24
2,Adelaide Hills,South Australia,2015-05-04,-34.876731,138.865255,29,26,6,82,54,1.9,1009,141,18,25
3,Adelaide Hills,South Australia,2015-05-07,-34.876731,138.865255,28,24,6,75,29,0.2,1011,127,13,20
4,Adelaide Hills,South Australia,2015-05-09,-34.876731,138.865255,28,25,7,78,55,0.0,1011,125,18,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1089,Yorke Peninsula,South Australia,2015-08-01,-34.690204,137.675221,27,24,6,76,54,0.3,1013,123,22,31
1090,Young,New South Wales,2015-08-05,-34.313500,148.301070,27,22,6,73,7,0.0,1012,123,14,22
1091,nil,South Australia,2015-12-30,-37.750000,145.033330,33,27,7,69,26,9.2,1008,275,17,25
1092,nil,South Australia,2016-10-23,-37.750000,145.033330,30,26,6,75,54,0.6,1009,132,17,23


#### Merging weather and fire data (based on weather observations)

In [48]:
weather['date'] = pd.to_datetime(weather.date)
weather_fire_reg = pd.merge(weather_grouped,fire_reg_grouped, how='left',left_on=['region','state','date'],right_on=['region','state','acq_date'])

weather_fire_reg

,region,state,date,lat_ave_x,long_ave_x,maxC,minC,uv,humidity,cloudcover,precipMM,pressure,windAZI,windKmph,WindGustKmph,acq_date,frp_min,frp_max,frp_sum,frp_median,frp_mean,lat_ave_y,long_ave_y
0,Adelaide Hills,South Australia,2015-05-01,-34.876731,138.865255,28,26,6,85,42,52.1,1009,144,18,30,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Adelaide Hills,South Australia,2015-05-02,-34.876731,138.865255,29,26,6,81,33,16.5,1009,129,15,24,2015-05-02,3.81,3.81,3.81,3.81,3.81,-34.95433,138.87901
2,Adelaide Hills,South Australia,2015-05-04,-34.876731,138.865255,29,26,6,82,54,1.9,1009,141,18,25,2015-05-04,2.22,2.22,2.22,2.22,2.22,-35.00000,138.75000
3,Adelaide Hills,South Australia,2015-05-07,-34.876731,138.865255,28,24,6,75,29,0.2,1011,127,13,20,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Adelaide Hills,South Australia,2015-05-09,-34.876731,138.865255,28,25,7,78,55,0.0,1011,125,18,25,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1089,Yorke Peninsula,South Australia,2015-08-01,-34.690204,137.675221,27,24,6,76,54,0.3,1013,123,22,31,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1090,Young,New South Wales,2015-08-05,-34.313500,148.301070,27,22,6,73,7,0.0,1012,123,14,22,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1091,nil,South Australia,2015-12-30,-37.750000,145.033330,33,27,7,69,26,9.2,1008,275,17,25,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1092,nil,South Australia,2016-10-23,-37.750000,145.033330,30,26,6,75,54,0.6,1009,132,17,23,2016-10-23,0.29,0.29,0.29,0.29,0.29,-27.70728,140.73697


#### Count of records with no data.
fire data null values are expected, fires do not occur everyday in every suburb

In [49]:
weather_fire_reg.isnull().sum()

region            0
state             0
date              0
lat_ave_x         0
long_ave_x        0
maxC              0
minC              0
uv                0
humidity          0
cloudcover        0
precipMM          0
pressure          0
windAZI           0
windKmph          0
WindGustKmph      0
acq_date        922
frp_min         922
frp_max         922
frp_sum         922
frp_median      922
frp_mean        922
lat_ave_y       922
long_ave_y      922
dtype: int64

#### Replace or drop missing values, sort, reorder & round 

In [50]:
# replacing 'acq_date' NaN values with corresponding values from 'date'
weather_fire_reg.acq_date.fillna(weather_fire_reg.date, inplace=True) 

# replacing frp NaN values with 0
weather_fire_reg.iloc[:,16:21] = weather_fire_reg.iloc[:,16:21].fillna(0)

# sort records and reorder columns
weather_fire_reg = weather_fire_reg.sort_values(['state','region','date'], ascending=[True,True,True]).reset_index(drop=True)
weather_fire_reg = weather_fire_reg[['date', 'state','region', 'lat_ave_x','long_ave_x','maxC', 'minC', 'uv', 'humidity','cloudcover', 'precipMM', 'pressure', 'windAZI', 'windKmph',
                                     'WindGustKmph', 'acq_date', 'frp_min', 'frp_max', 'frp_sum','frp_median', 'frp_mean']]

# dropping rows with region of 'nil'
weather_fire_reg = weather_fire_reg[weather_fire_reg.region !='nil']
# weather_fire_reg = weather_fire_reg.drop(['lat_ave_y','long_ave_y'],axis=1)

# rounding all weather and FRP values to 1 decimal place
cols = ['maxC', 'minC','uv', 'humidity', 'cloudcover', 'precipMM', 'pressure', 'windAZI','windKmph', 'WindGustKmph','frp_min', 'frp_max', 'frp_sum','frp_median', 'frp_mean']
weather_fire_reg[cols] = weather_fire_reg[cols].round(1)

weather_fire_reg

,date,state,region,lat_ave_x,long_ave_x,maxC,minC,uv,humidity,cloudcover,precipMM,pressure,windAZI,windKmph,WindGustKmph,acq_date,frp_min,frp_max,frp_sum,frp_median,frp_mean
0,2015-03-17,Australian Capital Territory,Australian Capital Territory,-35.331933,149.09838,30,26,6,80,63,4.6,1008,203,10,15,2015-03-17,0.6,1.0,2.8,0.6,0.7
1,2015-03-23,Australian Capital Territory,Australian Capital Territory,-35.331933,149.09838,29,26,6,80,53,27.4,1009,96,10,17,2015-03-23,0.0,0.0,0.0,0.0,0.0
2,2015-03-26,Australian Capital Territory,Australian Capital Territory,-35.331933,149.09838,29,26,6,81,43,1.4,1012,133,13,19,2015-03-26,1.1,6.1,30.4,1.9,2.8
3,2015-03-28,Australian Capital Territory,Australian Capital Territory,-35.331933,149.09838,29,25,6,78,33,1.3,1011,121,14,21,2015-03-28,0.4,36.0,212.5,6.3,11.2
4,2015-04-01,Australian Capital Territory,Australian Capital Territory,-35.331933,149.09838,29,27,6,82,56,18.6,1008,125,19,28,2015-04-01,0.8,23.5,74.3,3.3,6.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1089,2016-03-04,Western Australia,Waroona,-32.843230,115.92201,32,28,6,72,28,9.4,1009,228,5,8,2016-03-04,0.0,0.0,0.0,0.0,0.0
1090,2016-08-10,Western Australia,Wongan-Ballidu,-30.892940,116.71925,27,24,7,76,41,0.1,1014,117,20,29,2016-08-10,0.0,0.0,0.0,0.0,0.0
1091,2017-05-11,Western Australia,Wyndham-East Kimberley,-15.778130,128.74414,29,26,6,80,51,0.3,1011,134,20,29,2017-05-11,0.2,81.0,9539.2,4.8,7.0
1092,2016-10-22,Western Australia,Yalgoo,-28.183330,116.73333,30,27,6,78,81,2.2,1009,129,22,30,2016-10-22,0.0,0.0,0.0,0.0,0.0


In [51]:
# counts of records by state
weather_fire_reg['state'].value_counts()

New South Wales                 352
Victoria                        198
Queensland                      188
Western Australia               156
South Australia                 116
Tasmania                         60
Northern Territory               14
Australian Capital Territory      7
Name: state, dtype: int64

In [52]:
# check for missing values
weather_fire_reg.isnull().sum()

date            0
state           0
region          0
lat_ave_x       0
long_ave_x      0
maxC            0
minC            0
uv              0
humidity        0
cloudcover      0
precipMM        0
pressure        0
windAZI         0
windKmph        0
WindGustKmph    0
acq_date        0
frp_min         0
frp_max         0
frp_sum         0
frp_median      0
frp_mean        0
dtype: int64

### Export WEATHER_FIRE_REG

In [53]:
file = 'weather_fire_reg.csv'
vars()[file[0:-4]].to_csv('\\'.join([export_path,file[:]]),index = False)

---
---
---

## importing individual data files 
#### (for TESTING work)
#### - run import paths code first - first cell at top of page
change file name to import a different document

if there is date data in the .csv:
- specify the date columns names for date_fields variable(s)  - e.g. ['date', 'acq_date']

#### fire

In [ ]:
file, date_fields = 'fire.csv', ['date']
vars()[file[0:-4]] = pd.read_csv('\\'.join([import_path,file[:]]),parse_dates=date_fields)
print(file,' = ',vars()[file[0:-4]].shape)

#### weather

In [71]:
file, date_fields = 'weather.csv', ['date']
vars()[file[0:-4]] = pd.read_csv('\\'.join([import_path,file[:]]),parse_dates=date_fields)
print(file,' = ',vars()[file[0:-4]].shape)

weather.csv  =  (2084379, 20)


#### fire_weather

In [62]:
file, date_fields = 'fire_weather.csv', ['date','acq_date']
vars()[file[0:-4]] = pd.read_csv('\\'.join([import_path,file[:]]),parse_dates=date_fields)
print(file,' = ',vars()[file[0:-4]].shape)

fire_weather.csv  =  (6648806, 34)


#### weather_fire

In [64]:
file, date_fields = 'weather_fire.csv', ['date','acq_date']
vars()[file[0:-4]] = pd.read_csv('\\'.join([import_path,file[:]]),parse_dates=date_fields)
print(file,' = ',vars()[file[0:-4]].shape)

weather_fire.csv  =  (2054739, 31)


#### weather_fire_reg

In [2]:
file, date_fields = 'weather_fire_reg.csv', ['date','acq_date']
vars()[file[0:-4]] = pd.read_csv('\\'.join([import_path,file[:]]),parse_dates=date_fields)
print(file,' = ',vars()[file[0:-4]].shape)

weather_fire_reg.csv  =  (815031, 21)


#### fire_locats

In [65]:
file = 'fire_locats.csv'
vars()[file[0:-4]] = pd.read_csv('\\'.join([import_path,file[:]]))
print(file,' = ',vars()[file[0:-4]].shape)

fire_locats.csv  =  (1094, 5)


---
---